Notebook of spherical simulation data

In [1]:
import numpy as np
from scipy.linalg import sqrtm, inv
from scipy.stats import norm as stats_norm

def orth_mat(x):
    temp = x.T @ x 
    C = temp**(-1/2)
    return x*C

def gendata_sphere(n, p, rho=0, model='model1'):
 
    if p <= 5:
        raise ValueError("Dimension p must be greater than or equal to 6")
    if model not in ['model1', 'model2', 'model3']:
        raise ValueError("Model must be one of 'model1', 'model2', 'model3'")
    
   
    if rho > 0:
        x = np.zeros((n, p))
        x[:, 0] = np.random.normal(size=n) 
        for i in range(1, p):
            x[:, i] = rho * x[:, i - 1] + np.sqrt(1 - rho**2) * np.random.normal(size=n)
    else:
        x = np.random.normal(size=(n, p))  
    
   
    x = stats_norm.cdf(x)

    data = {}

    if model == 'model1':
        delta = np.random.normal(0, 0.2, size=n)  
        tmp = np.random.normal(0, 1, size=3)
        beta = np.concatenate((tmp, np.zeros(p - 3)))  
        beta = beta / np.sqrt(3)
        beta = beta.reshape(1, p)  
    
    
        x_beta = (x @ beta.T).flatten()  
    
        m_x = np.column_stack((np.cos(np.pi * x_beta), np.sin(np.pi * x_beta)))  
        eps = np.column_stack((-delta * np.sin(np.pi * x_beta), delta * np.cos(np.pi * x_beta)))  
        epsilon_norms = np.linalg.norm(eps, axis=1, keepdims=True)
        y = np.cos(epsilon_norms) * m_x + np.sin(epsilon_norms) * (eps / (epsilon_norms + 1e-8))
    
        data['x'] = x
        data['y'] = y
        data['d0'] = 1
        data['b0'] = beta

    
    elif model == 'model2':
        tmp1 = np.random.normal(0, 1, size=3)
        beta1 = np.concatenate((tmp1, np.zeros(p - 3))) 
        beta1 = beta1 / np.sqrt(3) 
        beta1 = beta1.reshape(1, p)  
        beta2 = np.concatenate(([0], [1], np.zeros(p - 2)))

    
        def genone(x, beta1):
            delta = np.random.normal(0, 0.2, size=2)
            x_beta1 = (x @ beta1.T).item() 
        
            m_x = np.array([
                np.sqrt(1 - (x[1])**2) * np.cos(np.pi * (x_beta1)),
                np.sqrt(1 - (x[1])**2) * np.sin(np.pi * (x_beta1)),
                (x[1])
                ])
            temp = np.array([[-np.sqrt(1 - (x[1])**2) * np.sin(np.pi * (x_beta1)), 
                          np.sqrt(1 - (x[1])**2) * np.cos(np.pi * (x_beta1)), 0]])
            temp = temp.flatten()
            v1 = orth_mat(temp)
            v2 = orth_mat(np.cross(v1, m_x))
            eps = delta[0] * v1 + delta[1] * v2
            y = np.cos(np.sqrt(np.sum(eps**2))) * m_x + np.sin(np.sqrt(np.sum(eps**2))) / np.sqrt(np.sum(eps**2)) * eps
            return np.concatenate((x, y))

        data_full = np.array([genone(x[i], beta1) for i in range(n)])
        data['x'] = data_full[:, :p]
        data['y'] = data_full[:, p:p + 3]
        data['d0'] = 2
        data['b0'] = np.vstack((beta1, beta2)).T

    
    elif model == 'model3':
        eps1 = np.random.normal(0, 0.2, size=n)  
    
    
        tmp = np.random.normal(0, 1, size=3)
        beta1 = np.concatenate((tmp, np.zeros(p - 3)))  
        beta1 = beta1 / np.sqrt(3)  
    
    
        tmp2 = np.random.normal(0, 1, size=3)
        beta2 = np.concatenate((np.zeros(p - 3), tmp2)) 
        beta2 = beta2 / np.sqrt(3)  
    
        eps2 = np.random.normal(0, 0.2, size=n)  
    
    
        m_x = np.column_stack((
            np.sin(np.pi * (x @ beta1)) * np.sin(np.pi * (x @ beta2)),
            np.sin(np.pi * (x @ beta1)) * np.cos(np.pi * (x @ beta2)),
            np.cos(np.pi * (x @ beta1))
            ))  
    

        y = np.column_stack((
            np.sin(np.pi * (x @ beta1) + eps1) * np.sin(np.pi * (x @ beta2) + eps2),
            np.sin(np.pi * (x @ beta1) + eps1) * np.cos(np.pi * (x @ beta2) + eps2),
            np.cos(np.pi * (x @ beta1) + eps1)
            )) 
    
    
        data['x'] = x 
        data['y'] = y 
        data['d0'] = 2
        data['b0'] = np.vstack((beta1, beta2)).T 


    return data




In [2]:

import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import LinAlgError
import time


n = 200
p = 10
rho = 0
model = 'model1'
num_repeats = 1
np.random.seed(123)
neigh = False
errors1 = np.zeros(num_repeats)
errors2 = np.zeros(num_repeats)
errors3 = np.zeros(num_repeats)

for repeat in range(num_repeats):
    data = gendata_sphere(n,p,rho,model)
    d_0 = data['d0']
    beta_true = data['b0']
    beta_true = beta_true.reshape(p,d_0)
    X =data['x']
    Y = data['y']

    metric = "Geodesic"
    if not neigh:
        # Neighborhood is unknown
        # Friedman et al, “Sparse inverse covariance estimation with the graphical lasso”, Biostatistics 9, pp 432, 2008
        from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
        glassomodel = GraphicalLassoCV()
        glassomodel.fit(X)
    
        omega = glassomodel.precision_
        np.where(np.sum(omega!=0, axis = 1) > 1)[0]

    Nb = []
    for i in range(p):
        Ni = (np.nonzero(omega[i,:])[0]).tolist() #np.nonzero(ome[i,:])[0]
        Nb.append(Ni)

    beta_gwire, _ = gwire_cv(X,Y,Nb,metric,d_0,fold=5)
    ygram = gram_matrix3(Y,1)
    beta_fopg = FOPG(X,ygram,d_0)
    init = beta_gwire
    ygram2 = gram_matrix3(Y,10)
    ygram2 = np.real(sqrtm(ygram2))
    beta_dcov, _,_  = MAVE1(X.T,ygram2,init)

    b1 = beta_gwire
    b2 = beta_dcov
    b3 = beta_fopg

    error2 = linalg_norm(b2 @ inv(b2.T @ b2) @ b2.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    error3 = linalg_norm(b3 @ inv(b3.T @ b3) @ b3.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    
    errors1[repeat] = error1
    errors2[repeat] = error2
    errors3[repeat] = error3

mean_errorG = np.mean(errors1)
sd_errorG = np.std(errors1)
mean_errorD = np.mean(errors2)
sd_errorD = np.std(errors2)
mean_errorF = np.mean(errors3)
sd_errorF = np.std(errors3)


print("GWIRE Mean Error:", mean_errorG)
print("GWIRE Standard Deviation of Error:", sd_errorG)

print("DCOV Mean Error:", mean_errorD)
print("DCOV Standard Deviation of Error:", sd_errorD)

print("FOPG Mean Error:", mean_errorF)
print("FOPG Standard Deviation of Error:", sd_errorF)

GWIRE Mean Error: 0.1414809572436286
GWIRE Standard Deviation of Error: 0.0
DCOV Mean Error: 0.06811315086565153
DCOV Standard Deviation of Error: 0.0
FOPG Mean Error: 0.06862560213878442
FOPG Standard Deviation of Error: 0.0


In [3]:

import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import LinAlgError

n = 400
p = 20
rho = 0
model = 'model1'
num_repeats = 100
np.random.seed(123)
neigh = False
errors1 = np.zeros(num_repeats)
errors2 = np.zeros(num_repeats)
errors3 = np.zeros(num_repeats)
Time1 = np.zeros(num_repeats)
Time2 = np.zeros(num_repeats)
Time3 = np.zeros(num_repeats)

for repeat in range(num_repeats):
    data = gendata_sphere(n,p,rho,model)
    d_0 = data['d0']
    beta_true = data['b0']
    beta_true = beta_true.reshape(p,d_0)
    X =data['x']
    Y = data['y']

    metric = "Geodesic"
    if not neigh:
        # Neighborhood is unknown
        # Friedman et al, “Sparse inverse covariance estimation with the graphical lasso”, Biostatistics 9, pp 432, 2008
        from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
        glassomodel = GraphicalLassoCV()
        glassomodel.fit(X)
    
        omega = glassomodel.precision_
        np.where(np.sum(omega!=0, axis = 1) > 1)[0]

    Nb = []
    for i in range(p):
        Ni = (np.nonzero(omega[i,:])[0]).tolist() #np.nonzero(ome[i,:])[0]
        Nb.append(Ni)
    start_time = time.time()
    beta_gwire, _ = gwire_cv(X,Y,Nb,metric,d_0,fold=5)
    gwire_time = time.time() - start_time
    Time1[repeat] = gwire_time
    ygram = gram_matrix3(Y,1)
    start_time = time.time()
    beta_fopg = FOPG(X,ygram,d_0)
    fopg_time = time.time() - start_time
    Time2[repeat] = fopg_time
    init = beta_gwire
    ygram2 = gram_matrix3(Y,10)
    ygram2 = np.real(sqrtm(ygram2))
    start_time = time.time()
    beta_dcov, _,_  = MAVE1(X.T,ygram2,init)
    mave1_time = time.time() - start_time
    Time3[repeat] = mave1_time

    b1 = beta_gwire
    b2 = beta_dcov
    b3 = beta_fopg

    error2 = linalg_norm(b2 @ inv(b2.T @ b2) @ b2.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    error3 = linalg_norm(b3 @ inv(b3.T @ b3) @ b3.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    
    errors1[repeat] = error1
    errors2[repeat] = error2
    errors3[repeat] = error3

mean_errorG = np.mean(errors1)
sd_errorG = np.std(errors1)
mean_errorD = np.mean(errors2)
sd_errorD = np.std(errors2)
mean_errorF = np.mean(errors3)
sd_errorF = np.std(errors3)

meantimeG = np.mean(Time1)
sdtimeG = np.std(Time1)
meantimeF = np.mean(Time2)
sdtimeF = np.std(Time2)
meantimeD = np.mean(Time3)
sdtimeD = np.std(Time3)

print("GWIRE Mean Error:", mean_errorG)
print("GWIRE Standard Deviation of Error:", sd_errorG)
print("GWIRE mean Time:", meantimeG)
print("GWIRE std of Time:", sdtimeG)

print("DCOV Mean Error:", mean_errorD)
print("DCOV Standard Deviation of Error:", sd_errorD)
print("DCOV mean Time",meantimeD)
print("DCOV std of Time:", sdtimeD)

print("FOPG Mean Error:", mean_errorF)
print("FOPG Standard Deviation of Error:", sd_errorF)
print("FOPG mean Time:",meantimeF)
print("FOPG std of Time:",sdtimeF)

GWIRE Mean Error: 0.2079599483756196
GWIRE Standard Deviation of Error: 0.09538326782437045
GWIRE mean Time: 3.382377049922943
GWIRE std of Time: 2.7917375666140027
DCOV Mean Error: 0.12146026659068519
DCOV Standard Deviation of Error: 0.05645846873638911
DCOV mean Time 0.0626603102684021
DCOV std of Time: 0.023431507438155034
FOPG Mean Error: 0.11208819780331548
FOPG Standard Deviation of Error: 0.04894213953311579
FOPG mean Time: 2.5615908455848695
FOPG std of Time: 0.20186842261101404


In [4]:

import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import LinAlgError

n = 200
p = 10
rho = 0.5
model = 'model1'
num_repeats = 1
np.random.seed(123)
neigh = False
errors1 = np.zeros(num_repeats)
errors2 = np.zeros(num_repeats)
errors3 = np.zeros(num_repeats)

for repeat in range(num_repeats):
    data = gendata_sphere(n,p,rho,model)
    d_0 = data['d0']
    beta_true = data['b0']
    beta_true = beta_true.reshape(p,d_0)
    X =data['x']
    Y = data['y']

    metric = "Geodesic"
    if not neigh:
        # Neighborhood is unknown
        # Friedman et al, “Sparse inverse covariance estimation with the graphical lasso”, Biostatistics 9, pp 432, 2008
        from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
        glassomodel = GraphicalLassoCV()
        glassomodel.fit(X)
    
        omega = glassomodel.precision_
        np.where(np.sum(omega!=0, axis = 1) > 1)[0]

    Nb = []
    for i in range(p):
        Ni = (np.nonzero(omega[i,:])[0]).tolist() #np.nonzero(ome[i,:])[0]
        Nb.append(Ni)

    beta_gwire, _ = gwire_cv(X,Y,Nb,metric,d_0,fold=5)
    ygram = gram_matrix3(Y,1)
    beta_fopg = FOPG(X,ygram,d_0)
    init = beta_gwire
    ygram2 = gram_matrix3(Y,10)
    ygram2 = np.real(sqrtm(ygram2))
    beta_dcov, _,_  = MAVE1(X.T,ygram2,init)

    b1 = beta_gwire
    b2 = beta_dcov
    b3 = beta_fopg

    error2 = linalg_norm(b2 @ inv(b2.T @ b2) @ b2.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    error3 = linalg_norm(b3 @ inv(b3.T @ b3) @ b3.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    
    errors1[repeat] = error1
    errors2[repeat] = error2
    errors3[repeat] = error3

mean_errorG = np.mean(errors1)
sd_errorG = np.std(errors1)
mean_errorD = np.mean(errors2)
sd_errorD = np.std(errors2)
mean_errorF = np.mean(errors3)
sd_errorF = np.std(errors3)


print("GWIRE Mean Error:", mean_errorG)
print("GWIRE Standard Deviation of Error:", sd_errorG)

print("DCOV Mean Error:", mean_errorD)
print("DCOV Standard Deviation of Error:", sd_errorD)

print("FOPG Mean Error:", mean_errorF)
print("FOPG Standard Deviation of Error:", sd_errorF)

GWIRE Mean Error: 0.4776697908498145
GWIRE Standard Deviation of Error: 0.0
DCOV Mean Error: 0.1408790540924726
DCOV Standard Deviation of Error: 0.0
FOPG Mean Error: 0.10991751230283264
FOPG Standard Deviation of Error: 0.0


In [5]:

import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import LinAlgError

n = 400
p = 20
rho = 0.5
model = 'model1'
num_repeats = 100
np.random.seed(123)
neigh = False
errors1 = np.zeros(num_repeats)
errors2 = np.zeros(num_repeats)
errors3 = np.zeros(num_repeats)
Time1 = np.zeros(num_repeats)
Time2 = np.zeros(num_repeats)
Time3 = np.zeros(num_repeats)

for repeat in range(num_repeats):
    data = gendata_sphere(n,p,rho,model)
    d_0 = data['d0']
    beta_true = data['b0']
    beta_true = beta_true.reshape(p,d_0)
    X =data['x']
    Y = data['y']

    metric = "Geodesic"
    if not neigh:
        # Neighborhood is unknown
        # Friedman et al, “Sparse inverse covariance estimation with the graphical lasso”, Biostatistics 9, pp 432, 2008
        from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
        glassomodel = GraphicalLassoCV()
        glassomodel.fit(X)
    
        omega = glassomodel.precision_
        np.where(np.sum(omega!=0, axis = 1) > 1)[0]

    Nb = []
    for i in range(p):
        Ni = (np.nonzero(omega[i,:])[0]).tolist() #np.nonzero(ome[i,:])[0]
        Nb.append(Ni)
    start_time = time.time()
    beta_gwire, _ = gwire_cv(X,Y,Nb,metric,d_0,fold=5)
    gwire_time = time.time() - start_time
    Time1[repeat] = gwire_time
    ygram = gram_matrix3(Y,1)
    start_time = time.time()
    beta_fopg = FOPG(X,ygram,d_0)
    fopg_time = time.time() - start_time
    Time2[repeat] = fopg_time
    init = beta_gwire
    ygram2 = gram_matrix3(Y,10)
    ygram2 = np.real(sqrtm(ygram2))
    start_time = time.time()
    beta_dcov, _,_  = MAVE1(X.T,ygram2,init)
    mave1_time = time.time() - start_time
    Time3[repeat] = mave1_time

    b1 = beta_gwire
    b2 = beta_dcov
    b3 = beta_fopg

    error2 = linalg_norm(b2 @ inv(b2.T @ b2) @ b2.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    error3 = linalg_norm(b3 @ inv(b3.T @ b3) @ b3.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    
    errors1[repeat] = error1
    errors2[repeat] = error2
    errors3[repeat] = error3

mean_errorG = np.mean(errors1)
sd_errorG = np.std(errors1)
mean_errorD = np.mean(errors2)
sd_errorD = np.std(errors2)
mean_errorF = np.mean(errors3)
sd_errorF = np.std(errors3)
meantimeG = np.mean(Time1)
sdtimeG = np.std(Time1)
meantimeF = np.mean(Time2)
sdtimeF = np.std(Time2)
meantimeD = np.mean(Time3)
sdtimeD = np.std(Time3)

print("GWIRE Mean Error:", mean_errorG)
print("GWIRE Standard Deviation of Error:", sd_errorG)
print("GWIRE mean Time:", meantimeG)
print("GWIRE std of Time:", sdtimeG)

print("DCOV Mean Error:", mean_errorD)
print("DCOV Standard Deviation of Error:", sd_errorD)
print("DCOV mean Time",meantimeD)
print("DCOV std of Time:", sdtimeD)

print("FOPG Mean Error:", mean_errorF)
print("FOPG Standard Deviation of Error:", sd_errorF)
print("FOPG mean Time:",meantimeF)
print("FOPG std of Time:",sdtimeF)

GWIRE Mean Error: 0.4882088557602576
GWIRE Standard Deviation of Error: 0.15242933502645425
GWIRE mean Time: 11.789179282188416
GWIRE std of Time: 0.3763089801783824
DCOV Mean Error: 0.1637066011956115
DCOV Standard Deviation of Error: 0.17245265387936295
DCOV mean Time 0.07385113954544067
DCOV std of Time: 0.07606310685605343
FOPG Mean Error: 0.15440767513445475
FOPG Standard Deviation of Error: 0.16477325788159483
FOPG mean Time: 2.516039447784424
FOPG std of Time: 0.054778797032673375


In [6]:

import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import LinAlgError

n = 200
p = 10
rho = 0
model = 'model2'
num_repeats = 1
np.random.seed(123)
neigh = False
errors1 = np.zeros(num_repeats)
errors2 = np.zeros(num_repeats)
errors3 = np.zeros(num_repeats)

for repeat in range(num_repeats):
    data = gendata_sphere(n,p,rho,model)
    d_0 = data['d0']
    beta_true = data['b0']
    beta_true = beta_true.reshape(p,d_0)
    X =data['x']
    Y = data['y']

    metric = "Geodesic"
    if not neigh:
        # Neighborhood is unknown
        # Friedman et al, “Sparse inverse covariance estimation with the graphical lasso”, Biostatistics 9, pp 432, 2008
        from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
        glassomodel = GraphicalLassoCV()
        glassomodel.fit(X)
    
        omega = glassomodel.precision_
        np.where(np.sum(omega!=0, axis = 1) > 1)[0]

    Nb = []
    for i in range(p):
        Ni = (np.nonzero(omega[i,:])[0]).tolist() #np.nonzero(ome[i,:])[0]
        Nb.append(Ni)

    beta_gwire, _ = gwire_cv(X,Y,Nb,metric,d_0,fold=5)
    ygram = gram_matrix3(Y,1)
    beta_fopg = FOPG(X,ygram,d_0)
    init = beta_gwire
    ygram2 = gram_matrix3(Y,10)
    ygram2 = np.real(sqrtm(ygram2))
    beta_dcov, _,_  = MAVE1(X.T,ygram2,init)

    b1 = beta_gwire
    b2 = beta_dcov
    b3 = beta_fopg

    error2 = linalg_norm(b2 @ inv(b2.T @ b2) @ b2.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    error3 = linalg_norm(b3 @ inv(b3.T @ b3) @ b3.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    
    errors1[repeat] = error1
    errors2[repeat] = error2
    errors3[repeat] = error3

mean_errorG = np.mean(errors1)
sd_errorG = np.std(errors1)
mean_errorD = np.mean(errors2)
sd_errorD = np.std(errors2)
mean_errorF = np.mean(errors3)
sd_errorF = np.std(errors3)


print("GWIRE Mean Error:", mean_errorG)
print("GWIRE Standard Deviation of Error:", sd_errorG)

print("DCOV Mean Error:", mean_errorD)
print("DCOV Standard Deviation of Error:", sd_errorD)

print("FOPG Mean Error:", mean_errorF)
print("FOPG Standard Deviation of Error:", sd_errorF)

GWIRE Mean Error: 0.4762672295796381
GWIRE Standard Deviation of Error: 0.0
DCOV Mean Error: 0.2721538856900268
DCOV Standard Deviation of Error: 0.0
FOPG Mean Error: 0.3136102650162096
FOPG Standard Deviation of Error: 0.0


In [7]:

import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import LinAlgError

n = 400
p = 20
rho = 0
model = 'model2'
num_repeats = 100
np.random.seed(123)
neigh = False
errors1 = np.zeros(num_repeats)
errors2 = np.zeros(num_repeats)
errors3 = np.zeros(num_repeats)
Time1 = np.zeros(num_repeats)
Time2 = np.zeros(num_repeats)
Time3 = np.zeros(num_repeats)

for repeat in range(num_repeats):
    data = gendata_sphere(n,p,rho,model)
    d_0 = data['d0']
    beta_true = data['b0']
    beta_true = beta_true.reshape(p,d_0)
    X =data['x']
    Y = data['y']

    metric = "Geodesic"
    if not neigh:
        # Neighborhood is unknown
        # Friedman et al, “Sparse inverse covariance estimation with the graphical lasso”, Biostatistics 9, pp 432, 2008
        from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
        glassomodel = GraphicalLassoCV()
        glassomodel.fit(X)
    
        omega = glassomodel.precision_
        np.where(np.sum(omega!=0, axis = 1) > 1)[0]

    Nb = []
    for i in range(p):
        Ni = (np.nonzero(omega[i,:])[0]).tolist() #np.nonzero(ome[i,:])[0]
        Nb.append(Ni)

    start_time = time.time()
    beta_gwire, _ = gwire_cv(X,Y,Nb,metric,d_0,fold=5)
    gwire_time = time.time() - start_time
    Time1[repeat] = gwire_time
    ygram = gram_matrix3(Y,1)
    start_time = time.time()
    beta_fopg = FOPG(X,ygram,d_0)
    fopg_time = time.time() - start_time
    Time2[repeat] = fopg_time
    init = beta_gwire
    ygram2 = gram_matrix3(Y,10)
    ygram2 = np.real(sqrtm(ygram2))
    start_time = time.time()
    beta_dcov, _,_  = MAVE1(X.T,ygram2,init)
    mave1_time = time.time() - start_time
    Time3[repeat] = mave1_time

    b1 = beta_gwire
    b2 = beta_dcov
    b3 = beta_fopg

    error2 = linalg_norm(b2 @ inv(b2.T @ b2) @ b2.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    error3 = linalg_norm(b3 @ inv(b3.T @ b3) @ b3.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    
    errors1[repeat] = error1
    errors2[repeat] = error2
    errors3[repeat] = error3

mean_errorG = np.mean(errors1)
sd_errorG = np.std(errors1)
mean_errorD = np.mean(errors2)
sd_errorD = np.std(errors2)
mean_errorF = np.mean(errors3)
sd_errorF = np.std(errors3)

meantimeG = np.mean(Time1)
sdtimeG = np.std(Time1)
meantimeF = np.mean(Time2)
sdtimeF = np.std(Time2)
meantimeD = np.mean(Time3)
sdtimeD = np.std(Time3)

print("GWIRE Mean Error:", mean_errorG)
print("GWIRE Standard Deviation of Error:", sd_errorG)
print("GWIRE mean Time:", meantimeG)
print("GWIRE std of Time:" ,sdtimeG)

print("DCOV Mean Error:", mean_errorD)
print("DCOV Standard Deviation of Error:", sd_errorD)
print("DCOV mean Time",meantimeD)
print("DCOV std of Time:", sdtimeD)

print("FOPG Mean Error:", mean_errorF)
print("FOPG Standard Deviation of Error:", sd_errorF)
print("FOPG mean Time:",meantimeF)
print("FOPG std of Time:",sdtimeF)

GWIRE Mean Error: 0.29706080643911253
GWIRE Standard Deviation of Error: 0.309943562617434
GWIRE mean Time: 3.0218319511413574
GWIRE std of Time: 2.3861865660927233
DCOV Mean Error: 0.3693699028658389
DCOV Standard Deviation of Error: 0.2793586498880498
DCOV mean Time 0.5410526514053344
DCOV std of Time: 0.666106663858487
FOPG Mean Error: 0.39138197102689304
FOPG Standard Deviation of Error: 0.3118651176388476
FOPG mean Time: 2.5308789515495302
FOPG std of Time: 0.08910165847654836


In [8]:

import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import LinAlgError

n = 200
p = 10
rho = 0.5
model = 'model2'
num_repeats = 1
np.random.seed(123)
neigh = False
errors1 = np.zeros(num_repeats)
errors2 = np.zeros(num_repeats)
errors3 = np.zeros(num_repeats)

for repeat in range(num_repeats):
    data = gendata_sphere(n,p,rho,model)
    d_0 = data['d0']
    beta_true = data['b0']
    beta_true = beta_true.reshape(p,d_0)
    X =data['x']
    Y = data['y']

    metric = "Geodesic"
    if not neigh:
        # Neighborhood is unknown
        # Friedman et al, “Sparse inverse covariance estimation with the graphical lasso”, Biostatistics 9, pp 432, 2008
        from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
        glassomodel = GraphicalLassoCV()
        glassomodel.fit(X)
    
        omega = glassomodel.precision_
        np.where(np.sum(omega!=0, axis = 1) > 1)[0]

    Nb = []
    for i in range(p):
        Ni = (np.nonzero(omega[i,:])[0]).tolist() #np.nonzero(ome[i,:])[0]
        Nb.append(Ni)

    beta_gwire, _ = gwire_cv(X,Y,Nb,metric,d_0,fold=5)
    ygram = gram_matrix3(Y,1)
    beta_fopg = FOPG(X,ygram,d_0)
    init = beta_gwire
    ygram2 = gram_matrix3(Y,10)
    ygram2 = np.real(sqrtm(ygram2))
    beta_dcov, _,_  = MAVE1(X.T,ygram2,init)

    b1 = beta_gwire
    b2 = beta_dcov
    b3 = beta_fopg

    error2 = linalg_norm(b2 @ inv(b2.T @ b2) @ b2.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    error3 = linalg_norm(b3 @ inv(b3.T @ b3) @ b3.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    
    errors1[repeat] = error1
    errors2[repeat] = error2
    errors3[repeat] = error3

mean_errorG = np.mean(errors1)
sd_errorG = np.std(errors1)
mean_errorD = np.mean(errors2)
sd_errorD = np.std(errors2)
mean_errorF = np.mean(errors3)
sd_errorF = np.std(errors3)


print("GWIRE Mean Error:", mean_errorG)
print("GWIRE Standard Deviation of Error:", sd_errorG)

print("DCOV Mean Error:", mean_errorD)
print("DCOV Standard Deviation of Error:", sd_errorD)

print("FOPG Mean Error:", mean_errorF)
print("FOPG Standard Deviation of Error:", sd_errorF)

GWIRE Mean Error: 0.7776860233566897
GWIRE Standard Deviation of Error: 0.0
DCOV Mean Error: 0.193443136379449
DCOV Standard Deviation of Error: 0.0
FOPG Mean Error: 0.2469521618093746
FOPG Standard Deviation of Error: 0.0


In [9]:

import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import LinAlgError

n = 400
p = 20
rho = 0.5
model = 'model2'
num_repeats = 100
np.random.seed(123)
neigh = False
errors1 = np.zeros(num_repeats)
errors2 = np.zeros(num_repeats)
errors3 = np.zeros(num_repeats)
Time1 = np.zeros(num_repeats)
Time2 = np.zeros(num_repeats)
Time3 = np.zeros(num_repeats)


for repeat in range(num_repeats):
    data = gendata_sphere(n,p,rho,model)
    d_0 = data['d0']
    beta_true = data['b0']
    beta_true = beta_true.reshape(p,d_0)
    X =data['x']
    Y = data['y']

    metric = "Geodesic"
    if not neigh:
        # Neighborhood is unknown
        # Friedman et al, “Sparse inverse covariance estimation with the graphical lasso”, Biostatistics 9, pp 432, 2008
        from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
        glassomodel = GraphicalLassoCV()
        glassomodel.fit(X)
    
        omega = glassomodel.precision_
        np.where(np.sum(omega!=0, axis = 1) > 1)[0]

    Nb = []
    for i in range(p):
        Ni = (np.nonzero(omega[i,:])[0]).tolist() #np.nonzero(ome[i,:])[0]
        Nb.append(Ni)

    start_time = time.time()
    beta_gwire, _ = gwire_cv(X,Y,Nb,metric,d_0,fold=5)
    gwire_time = time.time() - start_time
    Time1[repeat] = gwire_time
    ygram = gram_matrix3(Y,1)
    start_time = time.time()
    beta_fopg = FOPG(X,ygram,d_0)
    fopg_time = time.time() - start_time
    Time2[repeat] = fopg_time
    init = beta_gwire
    ygram2 = gram_matrix3(Y,10)
    ygram2 = np.real(sqrtm(ygram2))
    start_time = time.time()
    beta_dcov, _,_  = MAVE1(X.T,ygram2,init)
    mave1_time = time.time() - start_time
    Time3[repeat] = mave1_time

    b1 = beta_gwire
    b2 = beta_dcov
    b3 = beta_fopg

    error2 = linalg_norm(b2 @ inv(b2.T @ b2) @ b2.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    error3 = linalg_norm(b3 @ inv(b3.T @ b3) @ b3.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    
    errors1[repeat] = error1
    errors2[repeat] = error2
    errors3[repeat] = error3

mean_errorG = np.mean(errors1)
sd_errorG = np.std(errors1)
mean_errorD = np.mean(errors2)
sd_errorD = np.std(errors2)
mean_errorF = np.mean(errors3)
sd_errorF = np.std(errors3)


meantimeG = np.mean(Time1)
sdtimeG = np.std(Time1)
meantimeF = np.mean(Time2)
sdtimeF = np.std(Time2)
meantimeD = np.mean(Time3)
sdtimeD = np.std(Time3)

print("GWIRE Mean Error:", mean_errorG)
print("GWIRE Standard Deviation of Error:", sd_errorG)
print("GWIRE mean Time:", meantimeG)
print("GWIRE std of Time:", sdtimeG)

print("DCOV Mean Error:", mean_errorD)
print("DCOV Standard Deviation of Error:", sd_errorD)
print("DCOV mean Time",meantimeD)
print("DCOV std of Time:", sdtimeD)

print("FOPG Mean Error:", mean_errorF)
print("FOPG Standard Deviation of Error:", sd_errorF)
print("FOPG mean Time:",meantimeF)
print("FOPG std of Time:",sdtimeF)

GWIRE Mean Error: 0.6563097136096373
GWIRE Standard Deviation of Error: 0.2325294064957084
GWIRE mean Time: 12.03067106962204
GWIRE std of Time: 0.3283215195518172
DCOV Mean Error: 0.4100882384559597
DCOV Standard Deviation of Error: 0.23046382481843916
DCOV mean Time 0.5610763573646546
DCOV std of Time: 0.5348398565064999
FOPG Mean Error: 0.4572268498634194
FOPG Standard Deviation of Error: 0.2902613983259858
FOPG mean Time: 2.606202025413513
FOPG std of Time: 0.07362240715119216


In [10]:

import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import LinAlgError

n = 200
p = 10
rho = 0
model = 'model3'
num_repeats = 1
np.random.seed(123)
neigh = False
errors1 = np.zeros(num_repeats)
errors2 = np.zeros(num_repeats)
errors3 = np.zeros(num_repeats)

for repeat in range(num_repeats):
    data = gendata_sphere(n,p,rho,model)
    d_0 = data['d0']
    beta_true = data['b0']
    beta_true = beta_true.reshape(p,d_0)
    X =data['x']
    Y = data['y']

    metric = "Geodesic"
    if not neigh:
        # Neighborhood is unknown
        # Friedman et al, “Sparse inverse covariance estimation with the graphical lasso”, Biostatistics 9, pp 432, 2008
        from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
        glassomodel = GraphicalLassoCV()
        glassomodel.fit(X)
    
        omega = glassomodel.precision_
        np.where(np.sum(omega!=0, axis = 1) > 1)[0]

    Nb = []
    for i in range(p):
        Ni = (np.nonzero(omega[i,:])[0]).tolist() #np.nonzero(ome[i,:])[0]
        Nb.append(Ni)

    beta_gwire, _ = gwire_cv(X,Y,Nb,metric,d_0,fold=5)
    ygram = gram_matrix3(Y,1)
    beta_fopg = FOPG(X,ygram,d_0)
    init = beta_gwire
    ygram2 = gram_matrix3(Y,10)
    ygram2 = np.real(sqrtm(ygram2))
    beta_dcov, _,_  = MAVE1(X.T,ygram2,init)

    b1 = beta_gwire
    b2 = beta_dcov
    b3 = beta_fopg

    error2 = linalg_norm(b2 @ inv(b2.T @ b2) @ b2.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    error3 = linalg_norm(b3 @ inv(b3.T @ b3) @ b3.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    
    errors1[repeat] = error1
    errors2[repeat] = error2
    errors3[repeat] = error3

mean_errorG = np.mean(errors1)
sd_errorG = np.std(errors1)
mean_errorD = np.mean(errors2)
sd_errorD = np.std(errors2)
mean_errorF = np.mean(errors3)
sd_errorF = np.std(errors3)


print("GWIRE Mean Error:", mean_errorG)
print("GWIRE Standard Deviation of Error:", sd_errorG)

print("DCOV Mean Error:", mean_errorD)
print("DCOV Standard Deviation of Error:", sd_errorD)

print("FOPG Mean Error:", mean_errorF)
print("FOPG Standard Deviation of Error:", sd_errorF)

GWIRE Mean Error: 0.5149973654607235
GWIRE Standard Deviation of Error: 0.0
DCOV Mean Error: 0.14788589667145224
DCOV Standard Deviation of Error: 0.0
FOPG Mean Error: 0.18521498027927516
FOPG Standard Deviation of Error: 0.0


In [11]:

import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import LinAlgError

n = 400
p = 20
rho = 0
model = 'model3'
num_repeats = 100
np.random.seed(123)
neigh = False
errors1 = np.zeros(num_repeats)
errors2 = np.zeros(num_repeats)
errors3 = np.zeros(num_repeats)
Time1 = np.zeros(num_repeats)
Time2 = np.zeros(num_repeats)
Time3 = np.zeros(num_repeats)

for repeat in range(num_repeats):
    data = gendata_sphere(n,p,rho,model)
    d_0 = data['d0']
    beta_true = data['b0']
    beta_true = beta_true.reshape(p,d_0)
    X =data['x']
    Y = data['y']

    metric = "Geodesic"
    if not neigh:
        # Neighborhood is unknown
        # Friedman et al, “Sparse inverse covariance estimation with the graphical lasso”, Biostatistics 9, pp 432, 2008
        from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
        glassomodel = GraphicalLassoCV()
        glassomodel.fit(X)
    
        omega = glassomodel.precision_
        np.where(np.sum(omega!=0, axis = 1) > 1)[0]

    Nb = []
    for i in range(p):
        Ni = (np.nonzero(omega[i,:])[0]).tolist() #np.nonzero(ome[i,:])[0]
        Nb.append(Ni)

    start_time = time.time()
    beta_gwire, _ = gwire_cv(X,Y,Nb,metric,d_0,fold=5)
    gwire_time = time.time() - start_time
    Time1[repeat] = gwire_time
    ygram = gram_matrix3(Y,1)
    start_time = time.time()
    beta_fopg = FOPG(X,ygram,d_0)
    fopg_time = time.time() - start_time
    Time2[repeat] = fopg_time
    init = beta_gwire
    ygram2 = gram_matrix3(Y,10)
    ygram2 = np.real(sqrtm(ygram2))
    start_time = time.time()
    beta_dcov, _,_  = MAVE1(X.T,ygram2,init)
    mave1_time = time.time() - start_time
    Time3[repeat] = mave1_time

    b1 = beta_gwire
    b2 = beta_dcov
    b3 = beta_fopg

    error2 = linalg_norm(b2 @ inv(b2.T @ b2) @ b2.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    error3 = linalg_norm(b3 @ inv(b3.T @ b3) @ b3.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    
    errors1[repeat] = error1
    errors2[repeat] = error2
    errors3[repeat] = error3

mean_errorG = np.mean(errors1)
sd_errorG = np.std(errors1)
mean_errorD = np.mean(errors2)
sd_errorD = np.std(errors2)
mean_errorF = np.mean(errors3)
sd_errorF = np.std(errors3)


meantimeG = np.mean(Time1)
sdtimeG = np.std(Time1)
meantimeF = np.mean(Time2)
sdtimeF = np.std(Time2)
meantimeD = np.mean(Time3)
sdtimeD = np.std(Time3)

print("GWIRE Mean Error:", mean_errorG)
print("GWIRE Standard Deviation of Error:", sd_errorG)
print("GWIRE mean Time:", meantimeG)
print("GWIRE std of Time:", sdtimeG)

print("DCOV Mean Error:", mean_errorD)
print("DCOV Standard Deviation of Error:", sd_errorD)
print("DCOV mean Time",meantimeD)
print("DCOV std of Time:", sdtimeD)

print("FOPG Mean Error:", mean_errorF)
print("FOPG Standard Deviation of Error:", sd_errorF)
print("FOPG mean Time:",meantimeF)
print("FOPG std of Time:",sdtimeF)

GWIRE Mean Error: 0.6514410159514609
GWIRE Standard Deviation of Error: 0.44887058011982706
GWIRE mean Time: 3.478916115760803
GWIRE std of Time: 2.7513010596158076
DCOV Mean Error: 0.24757068677457345
DCOV Standard Deviation of Error: 0.15155534658780487
DCOV mean Time 0.45985093116760256
DCOV std of Time: 0.4642510524179688
FOPG Mean Error: 0.38456463456560613
FOPG Standard Deviation of Error: 0.371912543219503
FOPG mean Time: 2.629148497581482
FOPG std of Time: 0.1336866655692157


In [12]:

import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import LinAlgError

n = 200
p = 10
rho = 0.5
model = 'model3'
num_repeats = 1
np.random.seed(123)
neigh = False
errors1 = np.zeros(num_repeats)
errors2 = np.zeros(num_repeats)
errors3 = np.zeros(num_repeats)

for repeat in range(num_repeats):
    data = gendata_sphere(n,p,rho,model)
    d_0 = data['d0']
    beta_true = data['b0']
    beta_true = beta_true.reshape(p,d_0)
    X =data['x']
    Y = data['y']

    metric = "Geodesic"
    if not neigh:
        # Neighborhood is unknown
        # Friedman et al, “Sparse inverse covariance estimation with the graphical lasso”, Biostatistics 9, pp 432, 2008
        from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
        glassomodel = GraphicalLassoCV()
        glassomodel.fit(X)
    
        omega = glassomodel.precision_
        np.where(np.sum(omega!=0, axis = 1) > 1)[0]

    Nb = []
    for i in range(p):
        Ni = (np.nonzero(omega[i,:])[0]).tolist() #np.nonzero(ome[i,:])[0]
        Nb.append(Ni)

    beta_gwire, _ = gwire_cv(X,Y,Nb,metric,d_0,fold=5)
    ygram = gram_matrix3(Y,1)
    beta_fopg = FOPG(X,ygram,d_0)
    init = beta_gwire
    ygram2 = gram_matrix3(Y,10)
    ygram2 = np.real(sqrtm(ygram2))
    beta_dcov, _,_  = MAVE1(X.T,ygram2,init)

    b1 = beta_gwire
    b2 = beta_dcov
    b3 = beta_fopg

    error2 = linalg_norm(b2 @ inv(b2.T @ b2) @ b2.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    error3 = linalg_norm(b3 @ inv(b3.T @ b3) @ b3.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    
    errors1[repeat] = error1
    errors2[repeat] = error2
    errors3[repeat] = error3

mean_errorG = np.mean(errors1)
sd_errorG = np.std(errors1)
mean_errorD = np.mean(errors2)
sd_errorD = np.std(errors2)
mean_errorF = np.mean(errors3)
sd_errorF = np.std(errors3)


print("GWIRE Mean Error:", mean_errorG)
print("GWIRE Standard Deviation of Error:", sd_errorG)

print("DCOV Mean Error:", mean_errorD)
print("DCOV Standard Deviation of Error:", sd_errorD)

print("FOPG Mean Error:", mean_errorF)
print("FOPG Standard Deviation of Error:", sd_errorF)

GWIRE Mean Error: 0.7651163256357932
GWIRE Standard Deviation of Error: 0.0
DCOV Mean Error: 0.24084443229699773
DCOV Standard Deviation of Error: 0.0
FOPG Mean Error: 0.2628946410187407
FOPG Standard Deviation of Error: 0.0


In [13]:

import numpy as np
import scipy.linalg as la
from scipy.linalg import norm as linalg_norm
from lib_fun import *  
from MAVE1 import *
from gram_matrix import *
from FOPG import *
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import LinAlgError

n = 400
p = 20
rho = 0.5
model = 'model3'
num_repeats = 100
np.random.seed(123)
neigh = False
errors1 = np.zeros(num_repeats)
errors2 = np.zeros(num_repeats)
errors3 = np.zeros(num_repeats)
Time1 = np.zeros(num_repeats)
Time2 = np.zeros(num_repeats)
Time3 = np.zeros(num_repeats)

for repeat in range(num_repeats):
    data = gendata_sphere(n,p,rho,model)
    d_0 = data['d0']
    beta_true = data['b0']
    beta_true = beta_true.reshape(p,d_0)
    X =data['x']
    Y = data['y']

    metric = "Geodesic"
    if not neigh:
        # Neighborhood is unknown
        # Friedman et al, “Sparse inverse covariance estimation with the graphical lasso”, Biostatistics 9, pp 432, 2008
        from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
        glassomodel = GraphicalLassoCV()
        glassomodel.fit(X)
    
        omega = glassomodel.precision_
        np.where(np.sum(omega!=0, axis = 1) > 1)[0]

    Nb = []
    for i in range(p):
        Ni = (np.nonzero(omega[i,:])[0]).tolist() #np.nonzero(ome[i,:])[0]
        Nb.append(Ni)

    start_time = time.time()
    beta_gwire, _ = gwire_cv(X,Y,Nb,metric,d_0,fold=5)
    gwire_time = time.time() - start_time
    Time1[repeat] = gwire_time
    ygram = gram_matrix3(Y,1)
    start_time = time.time()
    beta_fopg = FOPG(X,ygram,d_0)
    fopg_time = time.time() - start_time
    Time2[repeat] = fopg_time
    init = beta_gwire
    ygram2 = gram_matrix3(Y,10)
    ygram2 = np.real(sqrtm(ygram2))
    start_time = time.time()
    beta_dcov, _,_  = MAVE1(X.T,ygram2,init)
    mave1_time = time.time() - start_time
    Time3[repeat] = mave1_time

    b1 = beta_gwire
    b2 = beta_dcov
    b3 = beta_fopg

    error2 = linalg_norm(b2 @ inv(b2.T @ b2) @ b2.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    error3 = linalg_norm(b3 @ inv(b3.T @ b3) @ b3.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    try:
    # Attempt to compute error3
        error1 = linalg_norm(b1 @ inv(b1.T @ b1) @ b1.T - beta_true @ inv(beta_true.T @ beta_true) @ beta_true.T, 'fro')
    except LinAlgError as e:
    
    # Optionally set error3 to some default value or handle as needed
        error1 = np.nan
    
    errors1[repeat] = error1
    errors2[repeat] = error2
    errors3[repeat] = error3

mean_errorG = np.mean(errors1)
sd_errorG = np.std(errors1)
mean_errorD = np.mean(errors2)
sd_errorD = np.std(errors2)
mean_errorF = np.mean(errors3)
sd_errorF = np.std(errors3)


meantimeG = np.mean(Time1)
sdtimeG = np.std(Time1)
meantimeF = np.mean(Time2)
sdtimeF = np.std(Time2)
meantimeD = np.mean(Time3)
sdtimeD = np.std(Time3)

print("GWIRE Mean Error:", mean_errorG)
print("GWIRE Standard Deviation of Error:", sd_errorG)
print("GWIRE mean Time:", meantimeG)
print("GWIRE std of Time:" ,sdtimeG)

print("DCOV Mean Error:", mean_errorD)
print("DCOV Standard Deviation of Error:", sd_errorD)
print("DCOV mean Time",meantimeD)
print("DCOV std of Time:", sdtimeD)

print("FOPG Mean Error:", mean_errorF)
print("FOPG Standard Deviation of Error:", sd_errorF)
print("FOPG mean Time:",meantimeF)
print("FOPG std of Time:",sdtimeF)

GWIRE Mean Error: 0.8987818912880479
GWIRE Standard Deviation of Error: 0.2791729898666676
GWIRE mean Time: 12.406458723545075
GWIRE std of Time: 0.5857868742104858
DCOV Mean Error: 0.27588789559147786
DCOV Standard Deviation of Error: 0.08869336994735154
DCOV mean Time 0.35535823106765746
DCOV std of Time: 0.2174841043499148
FOPG Mean Error: 0.4474532576262764
FOPG Standard Deviation of Error: 0.3624003624944691
FOPG mean Time: 2.7140277338027956
FOPG std of Time: 0.21603755647168213
